# Laboratorio 6 Data Science

El laboratorio consiste en realizar una serie de modificaciones al codigo visto en clase con el fin de lograr un mejor modelo. Por cada modificacion realizada al codigo en clase se anotaron ciertas modificaciones hacerca del comporatmiento general del modelo. Varias modificaciones se hacen alrededor del laboratorio y al final se modifica en general el codigo para lograr la mejor accuracy posible

## Codigo Visto en Clase con Modificaciones

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
datos_mnist, info_mnist = tfds.load(name='mnist',
                                    shuffle_files = False,
                                    with_info=True, 
                                    as_supervised=True)

In [3]:
entreno_mnist, prueba_mnist = datos_mnist['train'], datos_mnist['test']

In [4]:
num_obs_validacion = 0.1 * info_mnist.splits['train'].num_examples

In [5]:
num_obs_validacion = tf.cast(num_obs_validacion, tf.int64)

In [6]:
num_obs_prueba = info_mnist.splits['test'].num_examples

In [7]:
num_obs_prueba = tf.cast(num_obs_prueba, tf.int64)

In [8]:
def escalar(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32)
    imagen /= 255.
    return imagen, etiqueta

In [9]:
datos_entrenamiento_y_validacion_escalados = entreno_mnist.map(escalar)

In [10]:
datos_prueba = prueba_mnist.map(escalar)

In [11]:
TAMANIO_BUFFER = 10000

In [12]:
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_escalados.shuffle(TAMANIO_BUFFER)

In [13]:
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)

In [14]:
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [15]:
TAMANIO_TANDA = 10

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)

In [16]:
entradas_validacion, metas_validacion = next(iter(datos_validacion))

### Modelo

In [17]:
tamanio_entrada = 784
tamanio_salida = 10

In [18]:
tamanio_capa_escondida = 200 #Aqui se cambia el tamaño de la capa escondida por 200 o el mejor encontrado (500)

In [19]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    #Se modifica la funcion de activacion en el parametro de activation, puede ser relu, tanh o sigmoid
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida
    #Capas escondidas agregadas para el laboratorio
    #tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'),
    #tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'),
    #tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'),

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [20]:
#Aqui se modifica la tasa de aprendizaje
#Utilizar este optimizador envez de "adam" en la siguiente linea de codigo
#Utilizar cuando el tamaño de la tanda sea 1
optimizador_adhoc = tf.keras.optimizers.SGD(learning_rate=0.0001)

In [21]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
5400/5400 - 7s - loss: 0.1968 - accuracy: 0.9387 - val_loss: 0.1414 - val_accuracy: 0.9557
Epoch 2/5
5400/5400 - 6s - loss: 0.0953 - accuracy: 0.9709 - val_loss: 0.0711 - val_accuracy: 0.9767
Epoch 3/5
5400/5400 - 6s - loss: 0.0675 - accuracy: 0.9785 - val_loss: 0.0630 - val_accuracy: 0.9798
Epoch 4/5
5400/5400 - 6s - loss: 0.0528 - accuracy: 0.9836 - val_loss: 0.0621 - val_accuracy: 0.9800
Epoch 5/5
5400/5400 - 6s - loss: 0.0407 - accuracy: 0.9872 - val_loss: 0.0361 - val_accuracy: 0.9862


# Incisos del laboratorio

### 1. El ancho (tamaño de la capa escondida) del algoritmo. Intenten con un tamañode 200.  ¿Cómo cambia la precisión de validación del modelo?  ¿Cuánto tiempo se tardó el algoritmo en entrenar?  ¿Puede encontrar un tamaño de capa escondida que funcione mejor?

Con el valor de 200 se obtuvo una mejor precision siendo esta de 0.985, y se vio un pequeño cambio en el tiempo de ejecucion de todas las epocas. Se puede observar que para este caso mientras mas se aumente el tamaño de la capa escondida mas aumenta la presicion, sin embargo, el tiempo de ejecucion tambien es mas largo, finalmente se probo con un tamaño de 500 que obtuvo una presicion de 0.9882.

### 2. La profundidad del algoritmo.  Agreguen una capa escondida más al algoritmo. Este es un ejercicio extremadamente importante!  ¿Cómo cambia la precisión de validación?  ¿Qué hay del tiempo que se tarda en ejecutar?   Pista:  deben tener cuidado con las formas de los pesos y los sesgos.

Vemos que al agregar una capa escondida al modelo (siempre con relu) el valor de perdida del mismo tiende a disminuir mucho mas rapido que solo con 2 capas escondidas, tambien podemos ver un ligero aumento en la accuracy final del modelo. Finalmente se logra apreciar un ligero cambio en la ejecucion del entrenamiento.

### 3. El ancho y la profundidad del algoritmo.  Agregue cuantas capas sean necesarias para llegar a 5 capas escondidas.  Es más, ajusten el ancho del algoritmo conforme lo encuentre más conveniente.  ¿Cómo cambia la precisión de validación? ¿Qué hay del tiempo de ejecución?

Para este inciso se agregaron 3 nuevas capas escondidas con funcion de activacion Relu y se modifico el tamaño del ancho a 500. Apesar de unificar los mejores resultados de las modificaciones anteriores el modelo no logro sobre pasar el accuracy de 0.9882, que fue el obtenido al solo usar 2 capas esconidas y un ancho de 500. Sin embargo, el valor de perdida si se vio bastante afectado y el modelo aun es bastante alto y sobrepasa lo especificado por el laboratorio, adicionalmente al hacer esta prueba cada epoca tardo alrededor de 8 segundos, teniendo una duracion total de 40 segundos la mas tardada hasta el momento.

### 4. Experimenten con las funciones de activación.  Intenten aplicar una transformación sigmoidal a ambas capas.  La activación sigmoidal se obtiene escribiendo “sigmoid”.

Al modificar la funcion de activacion en ambas capas a sigmoid obtuvimos resultados mas bajos que al utilizar relu, en el caso de relu se obtuvo una presicion de 0.9718, mientras que con sigmoid se obtuvo una de 0.9615 bajando considerablemente.

### 5. Continúen experimentando con las funciones de activación.  Intenten aplicar un ReLu a la primera capa escondida y tanh a la segunda.  La activación tanh se obtiene escribiendo “tanh”.

Al mezclar ReLu con tanh se obtuvo un accuracy ligeramente mayor al obtenido solamente con sifmoid o relu, este siendo de 0.9737. En terminos de ejecucion es muy similar a todos los casos donde se utilizan 2 capas escondidas.

### 6. Ajusten el tamaño de la tanda.  Prueben con un tamaño de tanda de 10,000.  ¿Cómo cambia el tiempo requerido?  ¿Cómo cambia la precisión?
No se nota un cambio significativo en el tiempo de ejecucion del programa. Sin embargo, si se nota un cambio en la precision. Al cambiar el tamanio de tanda a 10,000 se obtiene una precision de 0.3 como la mas baja y 0.82 como la mas alta. Con un tamanio de 100 se obtiene una precision de 0.87 como la mas baja y 0.97 como la mas alta, mostrando un cambio bastante grande.

### 7. Ajusten el tamaño de la tanda a 1.  Eso corresponde al SGD. ¿Cómo cambian el tiempo y la precisión?  ¿Es el resultado coherente con la teoría?
Al ajustar el tamanio a 1, si se puede ver una gran diferencia en el tiempo de ejecucion siendo mucho mas tardado en comparacion a cuando se tiene una tanda de 100. En cuanto a la precision, la tanda de 1 muestra ser mas efectivo siendo 0.93 la precision mas baja y 0.98 la mas alta lo cual muestra ser cohernte con la teoria.

### 8. Ajusten la tasa de aprendizaje.  Prueben con un valor de 0.0001.  ¿Hace alguna diferencia?
No se ve una diferencia significativa a comparacion a una tasa de 0.001 en cuanto a tiempo de ejecucion ni con las precisiones. Cada epoch no tiene una diferencia de mas de 0.02 al comparar ambos valores.

### 9. Ajusten la tasa de aprendizaje a 0.02.  ¿Hay alguna diferencia?
De igual manera, no se ve una gran diferencia en tiempo de ejecucion ni en precisiones. Aunque esta vez la diferencia de precisiones mayor entre cada epoch es de 0.04.

### 10. Combinen todos los métodos indicados arriba e intenten llegar a una precisión de validación de 98.5% o más.
Al cambiar el tamanio de la capa escondida a 200, la tanda a 10, la tasa de crecimiento a 0.0001 se logro obtener una precision de 98.7%